#### Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import urllib

#### Function to pull assessment information

In [7]:
def assessments(state, year):
    """This function takes a state code and retrieves all the waterbody assessments for a specific reporting cycle.
    
    Parameters:
    state - two-letter state abbreviation
    year - 4-digit year you want data from"""
    
    url = 'https://attains.epa.gov/attains-public/api/assessments?state={}&reportingCycle={}'.format(state,year)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    
    assessments = []
    
    for i in range(0,len(data['items'][0]['assessments'])):
        water_id = data['items'][0]['assessments'][i]['assessmentUnitIdentifier']
        ir_category = data['items'][0]['assessments'][i]['epaIRCategory']
        assessment = [state, water_id, ir_category]
        assessments.append(assessment)
        
    df = pd.DataFrame(assessments, columns=['state', 'water_id', 'ir_category'])
    return df

#### Function to pull waterbody information

In [8]:
def waterbodies(state):
    """This function takes a state code and retrieves all the waterbodies for that state. The information returned includes 
    waterbody type, size, and units.
    
    Parameters:
    state - two-letter state abbreviation"""
    
    url_2 = 'https://attains.epa.gov/attains-public/api/assessmentUnits?stateCode={}'.format(state)
    response_2 = urllib.request.urlopen(url_2)
    data_2 = json.loads(response_2.read())
    
    waterbodies = []
    
    for i in range(0,len(data_2['items'][0]['assessmentUnits'])):
        water_id = data_2['items'][0]['assessmentUnits'][i]['assessmentUnitIdentifier']
        agency_code = data_2['items'][0]['assessmentUnits'][i]['agencyCode']
        status = data_2['items'][0]['assessmentUnits'][i]['statusIndicator']
        if len(data_2['items'][0]['assessmentUnits'][i]['waterTypes']) == 0:
            water_type = 'No Data'
            size = 'No Data'
            units = 'No Data'
        else:
            water_type = data_2['items'][0]['assessmentUnits'][i]['waterTypes'][0]['waterTypeCode']
            size = data_2['items'][0]['assessmentUnits'][i]['waterTypes'][0]['waterSizeNumber']
            units = data_2['items'][0]['assessmentUnits'][i]['waterTypes'][0]['unitsCode']
        water_body = [state, water_id, agency_code, status, water_type, size, units]
        waterbodies.append(water_body)
        
    df = pd.DataFrame(waterbodies, columns=['state', 'water_id', 'agency_code', 'status', 'type', 'size' , 'units'])
    return df

#### Combining both functions to create a joined dataframe for each state

#### Round 1

In [ ]:
#for data from 2018 reporting cycle
states = ['FL', 'LA', 'MI', 'MS', 'NC', 'OH']
state_dfs = []

for state in states:
    waters = waterbodies(state)
    categories = assessments(state, '2018')
    state_df = pd.merge(waters, categories, how='left', on=['state', 'water_id'])
    state_dfs.append(state_df)

combined_df = pd.concat(state_dfs)

In [132]:
#for data from 2016 reporting cycle
state = 'NJ'

waters = waterbodies(state)
categories = assessments(state, '2016')
new_jersey = pd.merge(waters, categories, how='left', on=['state', 'water_id'])

In [136]:
#for data from 2018/2020 combined reporting cycle
state = 'OR'

waters = waterbodies(state)
categories = assessments(state, '2020')
oregon = pd.merge(waters, categories, how='left', on=['state', 'water_id'])

In [137]:
#combining all data into one final df    
total_df = pd.concat([combined_df, new_jersey, oregon])

In [138]:
#writing to csv for future use
total_df.to_excel('Attains_Data.xlsx')

#### Round 2

In [4]:
#for data from 2018 reporting cycle
states = ['IN', 'IA', 'HI']
state_dfs = []

for state in states:
    waters = waterbodies(state)
    categories = assessments(state, '2018')
    state_df = pd.merge(waters, categories, how='left', on=['state', 'water_id'])
    state_dfs.append(state_df)

combined_df = pd.concat(state_dfs)

In [5]:
#combining all data into one final df    
total_df = combined_df

In [6]:
#writing to csv for future use
total_df.to_excel('Attains_Data_Round2.xlsx')

#### Round 3

In [9]:
state = 'AK'

waters = waterbodies(state)
categories = assessments(state, '2018')
alaska = pd.merge(waters, categories, how='left', on=['state', 'water_id'])

In [10]:
total_df = alaska

#### Impairment Analysis

In [3]:
#reading in data if in different session
total_df = pd.read_excel('Attains_Data_Round2.xlsx')
total_df.drop(columns=['Unnamed: 0'], inplace=True)

In [12]:
category_sums = total_df.groupby(['state', 'type', 'ir_category', 'units'])['size'].sum().reset_index()

In [14]:
category_sums[category_sums['state'] == 'AK']

,state,type,ir_category,units,size
0,AK,BAY,2,Square Miles,5.7616
1,AK,BAY,3,Square Miles,19471.3399
2,AK,BAY,4A,Square Miles,1.0024
3,AK,BAY,4B,Square Miles,0.0720
4,AK,BAY,5,Square Miles,0.0530
5,AK,BEACH,2,Miles,2.6000
6,AK,COASTAL,2,Miles,5.2800
7,AK,COASTAL,3,Miles,141.4500
8,AK,COASTAL,4B,Miles,14.0800
9,AK,CREEK,2,Miles,4.9400
